# Personalizer

*Personalizer* is a service that allows the application to chooise the best **single** content item to show to the user. Personalizer uses reinforcement learning techniques to enable the selection of the best item is based on the features about the content and the context. The selection is based on the collective behaviour and the reward score for all the users. 

The **Azure Personalizer** cognitive service in Microsoft Azure provides a cloud-based solution for creating personalized journeys for the users..

## Create a Personalizer resource

To use the Personalizer service, you need an Azure resource that you can use to train a model, and a resource with which you can publish it for applications to use. Unlike other congnitive services you will need to use one resource for every task you create. It would not be advisable to merge different content, or actions within the same resource. 

1. In a new browser tab, open the Azure portal at [https://portal.azure.com](https://portal.azure.com), and sign in using the Microsoft account associated with your Azure subscription.
2. Select the **&#65291;Create a resource** button, search for *personalizer*, and create a **Personalizer** resource with the following settings:
    - **Name**: *Enter a unique name*
    - **Subscription**: *Your Azure subscription*
    - **Location**: *Choose any available region*
    - **Pricing tier**: F0
    > **Note**: F0 tiers allows you about 50K transactions a month, that would be sufficient when you try out, but for all practical purposes you will need to setup a **S0** tier to be of any utility.

3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Overview** page, click the link to manage the keys for the service. You will need the endpoint and keys to connect to your cognitive services resource from client applications.

### Get the Key and Endpoint for your Personalizer resource

To use your cognitive services resource, client applications need its  endpoint and authentication key:

1. In the Azure portal, on the **Keys and Endpoint** page for your personalizer resource, copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_PER_KEY**.
2. Copy the **endpoint** for your resource and and paste it in the code below, replacing **YOUR_PER_ENDPOINT**.
3. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

In [ ]:
personalizer_key = 'YOUR_PER_KEY'
personalizer_endpoint = 'YOUR_PER_ENDPOINT'

print('Ready to use personalizer services at {} using key {}'.format(cog_endpoint, cog_key))

In [ ]:
import json
import matplotlib.pyplot as plt
import random
import requests
import time
import os
import uuid

In [ ]:
# Setup the APIs
personalization_rank_url = personalizer_endpoint + "/personalizer/v1.0/rank"
personalization_reward_url = personalizer_endpoint + "/personalizer/v1.0/events/"  
# add "{eventId}/reward"
headers = {'Ocp-Apim-Subscription-Key': personalizer_key, 'Content-Type': 'application/json'}

# Creating user features
Now we will create user features for which we will make the action recommendation for.
In our current case we are asking the user their choice for taste and the time of the day.

In [ ]:
#
# Feature files with attributes
# 
# The example file defines 4 users and thier choice of beverage depending on 
# the weather and time of the day
examplepath = os.path.join('data', 'example.json')
with open(examplepath) as handle:
    userpref = json.loads(handle.read())

In [ ]:
requestpath = os.path.join('data', "rankrequest.json")
with open(requestpath) as handle:
    rankactionsjsonobj = json.loads(handle.read())

In [ ]:
# Actions are the content items, such as news articles, specific movies, or products that the users
# can choose from. You can also define the attributes / features for each of the action. Unlike a 
# machine learning you dont have to define all the features for every action.
# In this example, we re defining the features of each coffee.
actionfeaturespath = os.path.join('data', "actionfeatures.json")
with open(actionfeaturespath) as handle:
    actionfeaturesobj = json.loads(handle.read())

In [ ]:
# Reward function that determines if the choice made by the personalizer is correct?
# In case of a correct choice, the reward is set to 1, else 0.
def get_reward_from_simulated_data(name, weather, timeofday, prediction):
    if userpref[name][weather][timeofday] == str(prediction):
        return 1
    return 0

In [ ]:
#
# Helper functions
def add_event_id(rankjsonobj):
    eventid = uuid.uuid4().hex
    rankjsonobj["eventId"] = eventid
    return eventid


def add_random_user_and_contextfeatures(namesoption, weatheropt, timeofdayopt, rankjsonobj):
    name = namesopt[random.randint(0,3)]
    weather = weatheropt[random.randint(0,2)]
    timeofday = timeofdayopt[random.randint(0,2)]
    rankjsonobj['contextFeatures'] = [{'timeofday': timeofday, 'weather': weather, 'name': name}]
    return [name, weather, timeofday]


def add_action_features(rankjsonobj):
    rankjsonobj["actions"] = actionfeaturesobj    

# Main Loop

In [ ]:
i = 1
num_requests = 1000
recommendations = 0
reward = 0
rewards = []
count = []
rankjsonobj = rankactionsjsonobj

namesopt = ['Alice', 'Bob', 'Cathy', 'Dave']
weatheropt = ['Sunny', 'Rainy', 'Snowy']
timeofdayopt = ['Morning', 'Afternoon', 'Evening']

In [ ]:
while (i <= num_requests):

    # create unique id to associate with an event
    eventid = add_event_id(rankjsonobj)

    # generate a random sample
    [name, weather, timeofday] = add_random_user_and_contextfeatures(namesopt, weatheropt, timeofdayopt, rankjsonobj)

    # add action features to rank
    add_action_features(rankjsonobj)

    # choose an action
    response = requests.post(personalization_rank_url, headers=headers, params=None, json=rankjsonobj)

    # compare personalization service recommendation with the simulated data to generate a reward value
    prediction = json.dumps(response.json()["rewardActionId"]).replace('"', '')
    reward = get_reward_from_simulated_data(name, weather, timeofday, prediction)

    # send the reward to the service
    response = requests.post(personalization_reward_url + eventid + "/reward", headers=headers, params=None,
                             json={"value": reward})

    # for every 10 rank request compute total correct recommendations
    recommendations = recommendations + reward

    # wait (>5 mins) between sending more events to observe learning in the next batch
    if (i % 500 == 0):
        time.sleep(310)

    if (i % 10 == 0):
        rewards.append(recommendations)
        count.append(i)
        recommendations = 0

    i = i + 1


In [ ]:
plt.plot(count, rewards)
plt.xlabel("Batch of 1000 rank events")
plt.ylabel("Correct recommendations per batch")
plt.show()